<a href="https://colab.research.google.com/github/wilfierd/Lab_AIN_01/blob/main/AIN_G2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import annotations
import itertools, random

class Sentence: pass
class Symbol(Sentence):
    def __init__(self, name): self.name = name
    def __repr__(self): return self.name
class Not(Sentence):
    def __init__(self, x): self.x = x
    def __repr__(self): return f"¬{self.x}"
class And(Sentence):
    def __init__(self, *xs): self.xs = xs
    def __repr__(self): return "(" + " ∧ ".join(map(str, self.xs)) + ")"
class Or(Sentence):
    def __init__(self, *xs): self.xs = xs
    def __repr__(self): return "(" + " ∨ ".join(map(str, self.xs)) + ")"

def eval_sentence(s: Sentence, m: dict) -> bool:
    if isinstance(s, Symbol): return bool(m.get(s.name, False))
    if isinstance(s, Not):    return not eval_sentence(s.x, m)
    if isinstance(s, And):    return all(eval_sentence(x, m) for x in s.xs)
    if isinstance(s, Or):     return any(eval_sentence(x, m) for x in s.xs)
    raise TypeError("Unknown sentence")

def model_check(KB: list[Sentence], query: Sentence, symbols: list[Symbol]) -> bool:
    names = [s.name for s in symbols]
    for vals in itertools.product([False, True], repeat=len(names)):
        m = dict(zip(names, vals))
        if all(eval_sentence(s, m) for s in KB):
            if not eval_sentence(query, m):
                return False
    return True

def exactly_one(symbols: list[Symbol]) -> list[Sentence]:
    axioms = [Or(*symbols)]
    for i in range(len(symbols)):
        for j in range(i+1, len(symbols)):
            axioms.append(Not(And(symbols[i], symbols[j])))
    return axioms

In [13]:
from __future__ import annotations

import sys
import shlex
import itertools
from typing import List, Dict


# ===== Domain =====
SUSPECTS = ["Lord Alaric", "Lady Morgana", "Butler Edwin"]
WEAPONS  = ["Silver Dagger", "Broken Wine Bottle", "Piano Wire"]
ROOMS    = ["Library", "Dining Hall", "Rose Garden"]


def build_symbols():
    S = {n: Symbol(f"S_{n}") for n in SUSPECTS}
    W = {n: Symbol(f"W_{n}") for n in WEAPONS}
    R = {n: Symbol(f"R_{n}") for n in ROOMS}
    ALL = list(S.values()) + list(W.values()) + list(R.values())
    return S, W, R, ALL


class LogicMystery:
    def __init__(self):
        self.S, self.W, self.R, self.ALL = build_symbols()
        self.KB: List = []
        # exactly-one constraints
        self.KB += exactly_one(list(self.S.values()))
        self.KB += exactly_one(list(self.W.values()))
        self.KB += exactly_one(list(self.R.values()))

    # ===== Utilities =====
    def add(self, *sentences):
        """Add facts, skipping duplicates"""
        existing = {repr(s) for s in self.KB}
        for s in sentences:
            if repr(s) not in existing:
                self.KB.append(s)
                existing.add(repr(s))

    def _has_model(self, KB) -> bool:
        """Check if there exists at least one satisfying model for KB"""
        names = [s.name for s in self.ALL]
        for vals in itertools.product([False, True], repeat=len(names)):
            m = dict(zip(names, vals))
            if all(eval_sentence(s, m) for s in KB):
                return True
        return False

    def consistent_with(self, *facts) -> bool:
        return self._has_model(self.KB + list(facts))

    def check(self):
        """Print YES (green) and MAYBE; hide NO (like CS50 style)."""
        if not self._has_model(self.KB):
            print("❌ KB inconsistent — no models satisfy current facts.")
            return
        GREEN = "\033[92m" if sys.stdout.isatty() else ""
        RESET = "\033[0m"  if sys.stdout.isatty() else ""
        for sym in self.ALL:
            if model_check(self.KB, sym, self.ALL):
                print(f"{GREEN}{sym}: YES{RESET}")
            elif not model_check(self.KB, Not(sym), self.ALL):
                print(f"{sym}: MAYBE")

    def candidates(self):
        """Enumerate consistent models → (suspect, weapon, room)"""
        names = [s.name for s in self.ALL]
        triples = set()
        for vals in itertools.product([False, True], repeat=len(names)):
            m = dict(zip(names, vals))
            if all(eval_sentence(s, m) for s in self.KB):
                s = next(k[2:] for k, v in m.items() if k.startswith("S_") and v)
                w = next(k[2:] for k, v in m.items() if k.startswith("W_") and v)
                r = next(k[2:] for k, v in m.items() if k.startswith("R_") and v)
                triples.add((s, w, r))
        return sorted(triples)

    def forced_solution(self):
        """Return unique solution if KB forces exactly one tuple"""
        cand = self.candidates()
        if not cand:
            return None
        Sset = {s for s, _, _ in cand}
        Wset = {w for _, w, _ in cand}
        Rset = {r for _, _, r in cand}
        if len(Sset) == 1 and len(Wset) == 1 and len(Rset) == 1:
            return (next(iter(Sset)), next(iter(Wset)), next(iter(Rset)))
        return None

    # ===== Command helpers =====
    def exclude(self, kind: str, name: str):
        d = {"s": self.S, "w": self.W, "r": self.R}[kind]
        target = self._resolve_name(d, name)
        if not target:
            print("?? no match"); return
        fact = Not(target)

        if any(repr(fact) == repr(s) for s in self.KB):
            print(f"(already had ¬{target})"); return
        if not self.consistent_with(fact):
            print(f"❌ rejecting: adding ¬{target} would make KB inconsistent")
            return

        self.add(fact)
        print(f"added: ¬{target}")

    def only(self, kind: str, name: str):
        d = {"s": self.S, "w": self.W, "r": self.R}[kind]
        target = self._resolve_name(d, name)
        if not target:
            print("?? no match"); return
        fact = target

        if any(repr(fact) == repr(s) for s in self.KB):
            print(f"(already had {target})"); return
        if not self.consistent_with(fact):
            print(f"❌ rejecting: adding {target} would make KB inconsistent")
            return

        self.add(fact)
        print(f"added: {target}")

    def _resolve_name(self, d: Dict[str, Symbol], text: str):
        t = text.lower()
        # exact match
        for n, sym in d.items():
            if n.lower() == t:
                return sym
        # substring match
        options = [sym for n, sym in d.items() if t in n.lower()]
        if len(options) == 1:
            return options[0]
        if options:
            print("ambiguous, be more specific:")
            for n, sym in d.items():
                if t in n.lower():
                    print(" -", n)
        return None


HELP = """
Commands
  help                         : show this
  list                         : list domain (suspects / weapons / rooms)
  status                       : print YES/MAYBE (NO is hidden)
  cand                         : print remaining (Suspect, Weapon, Room) tuples
  solve                        : print solution if KB forces a unique one
  s.no <name...>               : exclude suspects (comma separated ok)
  w.no <name...>               : exclude weapons
  r.no <name...>               : exclude rooms
  s.yes <name...>              : assert suspect
  w.yes <name...>              : assert weapon
  r.yes <name...>              : assert room
  quit / exit                  : leave

Tips
- You can pass multiple names:  s.no Alaric, Edwin
- Partial names are fine:       w.no wire
""".strip()


def list_domain():
    print("Suspects:", ", ".join(SUSPECTS))
    print("Weapons :", ", ".join(WEAPONS))
    print("Rooms   :", ", ".join(ROOMS))


def parse_multi_names(argv):
    raw = " ".join(argv)
    parts = [p.strip() for p in raw.split(",") if p.strip()]
    seen, out = set(), []
    for p in parts:
        key = p.lower()
        if key not in seen:
            seen.add(key)
            out.append(p)
    return out


def main():
    g = LogicMystery()
    print("🔎 THE MANSION MURDER — logic-only shell")
    list_domain()
    print(HELP)

    while True:
        try:
            raw = input("> ").strip()
        except (EOFError, KeyboardInterrupt):
            print(); break
        if not raw:
            continue

        args = shlex.split(raw)
        cmd = args[0].lower()

        if cmd in ("quit", "exit"):
            break
        elif cmd == "help":
            print(HELP)
        elif cmd == "list":
            list_domain()
        elif cmd == "status":
            g.check()
        elif cmd == "cand":
            cand = g.candidates()
            print(f"{len(cand)} candidates:")
            for t in cand:
                print(" -", t)
        elif cmd == "solve":
            ans = g.forced_solution()
            if ans:
                print("✅ forced by logic:", {"Culprit": ans[0], "Weapon": ans[1], "Scene": ans[2]})
            else:
                print("…not enough constraints yet.")
        elif cmd in ("s.no", "w.no", "r.no", "s.yes", "w.yes", "r.yes"):
            if len(args) < 2:
                print("missing names; see `help`."); continue
            kind = cmd[0]  # 's' | 'w' | 'r'
            positive = cmd.endswith(".yes")
            for name in parse_multi_names(args[1:]):
                if positive:
                    g.only(kind, name)
                else:
                    g.exclude(kind, name)
        else:
            print("unknown command; see `help`.")


if __name__ == "__main__":
    main()


🔎 THE MANSION MURDER — logic-only shell
Suspects: Lord Alaric, Lady Morgana, Butler Edwin
Weapons : Silver Dagger, Broken Wine Bottle, Piano Wire
Rooms   : Library, Dining Hall, Rose Garden
Commands
  help                         : show this
  list                         : list domain (suspects / weapons / rooms)
  status                       : print YES/MAYBE (NO is hidden)
  cand                         : print remaining (Suspect, Weapon, Room) tuples
  solve                        : print solution if KB forces a unique one
  s.no <name...>               : exclude suspects (comma separated ok)
  w.no <name...>               : exclude weapons
  r.no <name...>               : exclude rooms
  s.yes <name...>              : assert suspect
  w.yes <name...>              : assert weapon
  r.yes <name...>              : assert room
  quit / exit                  : leave

Tips
- You can pass multiple names:  s.no Alaric, Edwin
- Partial names are fine:       w.no wire
> cand
27 candidates: